# core

> This module contains all the core functions used in the library.

In [ ]:
#| default_exp core

In [ ]:
#| export
import logging
import os

from datasets import concatenate_datasets, Dataset
from rich.logging import RichHandler

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logger.addHandler(RichHandler(rich_tracebacks=True))
# Turn off logging for datasets
logging.getLogger("datasets").setLevel(logging.ERROR)

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
from datasets import load_dataset
from squeakily.filter import check_char_repetition, check_flagged_words, minhash_dedup
from squeakily.clean import remove_empty_lines, normalize_whitespace

In [ ]:
#| export
class Pipeline:
    """
    A pipeline is a collection of datasources and their associated transformations to be run.
    """
    def __init__(
        self,
        datasources # The datasources to be run
    ):
        self.datasources = datasources
    
    def run(
        self,
        global_filters=[],      # Filters to be run at the dataset level rather than the example level
        global_cleaners=[],     # Cleaners to be run at the dataset level rather than the example level
        cleaning_first=False,   # Whether to run the cleaning transformations first
        globals_first=False,    # Whether to run the global transformations first
        dry_run=False,          # Whether to run the pipeline or only calculate the various criteria and add as a column
    ):
        """
        Run the pipeline.
        """
        for i in range(len(self.datasources)):
            column = self.datasources[i]["columns"][0]
            logger.info(f"Running datasource: {self.datasources[i]['dataset'].builder_name}")
            if cleaning_first:
                for c in self.datasources[i]["cleaners"]:
                    name = c.__name__
                    logger.info(f"Running cleaner: {name} on {column}")
                    self.datasources[i]["dataset"] = self.datasources[i]["dataset"].map(
                        lambda x: {column: c(x[column])},
                        num_proc=os.cpu_count(),
                    )
                for f in self.datasources[i]["filters"]:
                    name = f.__name__
                    logger.info(f"Running filter: {name} on {column}")
                    if dry_run:
                        logger.info(f"Running in dry-run mode")
                        self.datasources[i]["dataset"] = self.datasources[i]["dataset"].map(
                            lambda x: {f"{name}_criteria": f(x[column], dry_run=True)},
                            num_proc=os.cpu_count(),
                        )
                    else:
                        self.datasources[i]["dataset"] = self.datasources[i]["dataset"].filter(
                            lambda x: f(x[column]),
                            num_proc=os.cpu_count(),
                        )
            else:
                for f in self.datasources[i]["filters"]:
                    name = f.__name__
                    logger.info(f"Running filter: {name} on {column}")
                    if dry_run:
                        logger.info(f"Running in dry-run mode")
                        self.datasources[i]["dataset"] = self.datasources[i]["dataset"].map(
                            lambda x: {f"{name}_criteria": f(x[column], dry_run=True)},
                            num_proc=os.cpu_count(),
                        )
                    else:
                        self.datasources[i]["dataset"] = self.datasources[i]["dataset"].filter(
                            lambda x: f(x[column]),
                            num_proc=os.cpu_count(),
                        )
                for c in self.datasources[i]["cleaners"]:
                    name = c.__name__
                    logger.info(f"Running cleaner: {name} on {column}")
                    self.datasources[i]["dataset"] = self.datasources[i]["dataset"].map(
                        lambda x: {column: c(x[column])},
                        num_proc=os.cpu_count(),
                    )

        if len(global_filters) > 0:
            # concatenate all datasets
            datasets = [
                d["dataset"] for d in self.datasources
                if not d.get("skip_global", False)
            ]
            global_column = self.datasources[0]["columns"][0]
            global_dataset = concatenate_datasets(datasets)

            # Add a column representing the original dataset name
            md = []
            for d in datasets:
                md.extend([d.builder_name] * len(d))
            meta_data = Dataset.from_dict({"meta_data": md})
            global_dataset_with_meta = concatenate_datasets([global_dataset, meta_data], axis=1)

            # Run the global filters
            for f in global_filters:
                logger.info(f"Running global filter: {f.__name__}")
                global_dataset_with_meta = f(global_dataset_with_meta, global_column, dry_run=dry_run)

            # Split the dataset back up
            for i, dataset in enumerate(datasets):
                self.datasources[i]["dataset"] = global_dataset_with_meta.filter(
                    lambda x: x["meta_data"] == dataset.builder_name,
                    num_proc=os.cpu_count(),
                )

In [ ]:
#|echo: true
show_doc(Pipeline.run)

---

[source](https://github.com/CarperAI/squeakily/blob/main/squeakily/core.py#L30){target="_blank" style="float:right; font-size:smaller"}

### Pipeline.run

>      Pipeline.run (global_filters=[], global_cleaners=[],
>                    cleaning_first=False, globals_first=False, dry_run=False)

Run the pipeline.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| global_filters | list | [] | Filters to be run at the dataset level rather than the example level |
| global_cleaners | list | [] | Cleaners to be run at the dataset level rather than the example level |
| cleaning_first | bool | False | Whether to run the cleaning transformations first |
| globals_first | bool | False | Whether to run the global transformations first |
| dry_run | bool | False | Whether to run the pipeline or only calculate the various criteria and add as a column |

In [ ]:
# test dry run
ds = load_dataset("wikitext", "wikitext-103-v1", split="train[:1%]")
logger.info(f"Original dataset size: {len(ds)}")
datasources = [
    {
        "dataset": ds,
        "columns": ["text"],
        "filters": [check_char_repetition, check_flagged_words],
        "cleaners": [remove_empty_lines, normalize_whitespace],
    },
    # ...
]

pipeline = Pipeline(datasources)
pipeline.run(dry_run=True, global_filters=[minhash_dedup])

assert len(ds) == len(pipeline.datasources[0]["dataset"])
assert "check_char_repetition_criteria" in pipeline.datasources[0]["dataset"].features
assert "check_flagged_words_criteria" in pipeline.datasources[0]["dataset"].features
assert "duplicate" in pipeline.datasources[0]["dataset"].features
assert "meta_data" in pipeline.datasources[0]["dataset"].features
assert "__id__" in pipeline.datasources[0]["dataset"].features

[12/01/22 20:44:47] INFO     Original dataset size: 18014                                           ]8;id=671960;file:///tmp/ipykernel_3328129/1067455855.py\1067455855.py]8;;\:]8;id=749686;file:///tmp/ipykernel_3328129/1067455855.py#3\3]8;;\

                    INFO     Running datasource: wikitext                                          ]8;id=265449;file:///tmp/ipykernel_3328129/1832331928.py\1832331928.py]8;;\:]8;id=880494;file:///tmp/ipykernel_3328129/1832331928.py#25\25]8;;\

                    INFO     Running filter: check_char_repetition on text                         ]8;id=903622;file:///tmp/ipykernel_3328129/1832331928.py\1832331928.py]8;;\:]8;id=201214;file:///tmp/ipykernel_3328129/1832331928.py#51\51]8;;\

                    INFO     Running in dry-run mode                                               ]8;id=194074;file:///tmp/ipykernel_3328129/1832331928.py\1832331928.py]8;;\:]8;id=915580;file:///tmp/ipykernel_3328129/1832331928.py#53\53]8;;\

                    INFO     Running filter: check_flagged_words on text                           ]8;id=422119;file:///tmp/ipykernel_3328129/1832331928.py\1832331928.py]8;;\:]8;id=887209;file:///tmp/ipykernel_3328129/1832331928.py#51\51]8;;\

...

#25:   0%|          | 0/1 [00:00<?, ?ba/s]

#29:   0%|          | 0/1 [00:00<?, ?ba/s]

#30:   0%|          | 0/1 [00:00<?, ?ba/s]

#24:   0%|          | 0/1 [00:00<?, ?ba/s]

#31:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
# test dry run with partials
from functools import partial

ds = load_dataset("wikitext", "wikitext-103-v1", split="train[:1%]")
logger.info(f"Original dataset size: {len(ds)}")
check_char_repetition_p = partial(check_char_repetition, char_repetition_len=3)
check_char_repetition_p.__name__ = "check_char_repetition"
datasources = [
    {
        "dataset": ds,
        "columns": ["text"],
        "filters": [check_char_repetition_p, check_flagged_words],
        "cleaners": [remove_empty_lines, normalize_whitespace],
    },
    # ...
]

pipeline = Pipeline(datasources)
pipeline.run(dry_run=True)

assert len(ds) == len(pipeline.datasources[0]["dataset"])
assert "check_char_repetition_criteria" in pipeline.datasources[0]["dataset"].features
assert "check_flagged_words_criteria" in pipeline.datasources[0]["dataset"].features

[12/01/22 20:48:28] INFO     Original dataset size: 18014                                           ]8;id=674399;file:///tmp/ipykernel_3328129/3559480091.py\3559480091.py]8;;\:]8;id=506592;file:///tmp/ipykernel_3328129/3559480091.py#5\5]8;;\

                    INFO     Running datasource: wikitext                                          ]8;id=829563;file:///tmp/ipykernel_3328129/1832331928.py\1832331928.py]8;;\:]8;id=37693;file:///tmp/ipykernel_3328129/1832331928.py#25\25]8;;\

                    INFO     Running filter: check_char_repetition on text                         ]8;id=1675;file:///tmp/ipykernel_3328129/1832331928.py\1832331928.py]8;;\:]8;id=683864;file:///tmp/ipykernel_3328129/1832331928.py#51\51]8;;\

                    INFO     Running in dry-run mode                                               ]8;id=12179;file:///tmp/ipykernel_3328129/1832331928.py\1832331928.py]8;;\:]8;id=295793;file:///tmp/ipykernel_3328129/1832331928.py#53\53]8;;\

#0:   0%|          | 0/563 [00:00<?, ?ex/s]

...

#27:   0%|          | 0/563 [00:00<?, ?ex/s]

#28:   0%|          | 0/563 [00:00<?, ?ex/s]

#29:   0%|          | 0/563 [00:00<?, ?ex/s]

#30:   0%|          | 0/562 [00:00<?, ?ex/s]

#31:   0%|          | 0/562 [00:00<?, ?ex/s]

In [ ]:
ds = load_dataset("wikitext", "wikitext-103-v1", split="train[:1%]")
logger.info(f"Original dataset size: {len(ds)}")
datasources = [
    {
        "dataset": ds,
        "columns": ["text"],
        "filters": [check_char_repetition, check_flagged_words],
        "cleaners": [remove_empty_lines, normalize_whitespace],
    },
    # ...
]

global_filters = [minhash_dedup]
pipeline = Pipeline(datasources)
pipeline.run(global_filters=global_filters)
logger.info(f"Final dataset size: {len(pipeline.datasources[0]['dataset'])}")

assert len(ds) > len(pipeline.datasources[0]["dataset"])

[12/01/22 20:48:45] INFO     Original dataset size: 18014                                            ]8;id=976064;file:///tmp/ipykernel_3328129/131731803.py\131731803.py]8;;\:]8;id=699481;file:///tmp/ipykernel_3328129/131731803.py#2\2]8;;\

                    INFO     Running datasource: wikitext                                          ]8;id=292657;file:///tmp/ipykernel_3328129/1832331928.py\1832331928.py]8;;\:]8;id=45601;file:///tmp/ipykernel_3328129/1832331928.py#25\25]8;;\

                    INFO     Running filter: check_char_repetition on text                         ]8;id=663003;file:///tmp/ipykernel_3328129/1832331928.py\1832331928.py]8;;\:]8;id=632446;file:///tmp/ipykernel_3328129/1832331928.py#51\51]8;;\

                    INFO     Running filter: check_flagged_words on text                           ]8;id=201251;file:///tmp/ipykernel_3328129/1832331928.py\1832331928.py]8;;\:]8;id=603796;file:///tmp/ipykernel_3328129/1832331928.py#51\51]8;;\

                    INFO     Running cleaner: remove_empty_lines on text                           ]8;id=8364;file:///tmp/ipykernel_3328129/1832331928.py\1832331928.py]8;;\:]8;id=656682;file:///tmp/ipykernel_3328129/1832331928.py#65\65]8;;\

...

#28:   0%|          | 0/1 [00:00<?, ?ba/s]

#29:   0%|          | 0/1 [00:00<?, ?ba/s]

#30:   0%|          | 0/1 [00:00<?, ?ba/s]

#31:   0%|          | 0/1 [00:00<?, ?ba/s]

[12/01/22 20:49:38] INFO     Final dataset size: 10557                                              ]8;id=559915;file:///tmp/ipykernel_3328129/131731803.py\131731803.py]8;;\:]8;id=836424;file:///tmp/ipykernel_3328129/131731803.py#16\16]8;;\

In [ ]:
# test the ability to skip global filters
ds_1 = load_dataset("wikitext", "wikitext-103-v1", split="train[:1%]")

datasources = [
    {
        "dataset": ds,
        "columns": ["text"],
        "filters": [check_char_repetition, check_flagged_words],
        "cleaners": [remove_empty_lines, normalize_whitespace],
        "skip_global": False,
    },
    {
        "dataset": ds_1,
        "columns": ["text"],
        "filters": [check_char_repetition, check_flagged_words],
        "cleaners": [remove_empty_lines, normalize_whitespace],
        "skip_global": True,
    },
    # ...
]
pipeline = Pipeline(datasources)
pipeline.run(global_filters=global_filters)
logger.info(f"Final dataset size: {len(pipeline.datasources[0]['dataset'])}")

assert len(pipeline.datasources[0]["dataset"]) < len(pipeline.datasources[1]["dataset"])

[11/30/22 01:41:53] INFO     Running datasource: wikitext                                          ]8;id=179809;file:///tmp/ipykernel_72012/1743738010.py\1743738010.py]8;;\:]8;id=952882;file:///tmp/ipykernel_72012/1743738010.py#25\25]8;;\

                    INFO     Running filter: check_char_repetition on text                         ]8;id=740765;file:///tmp/ipykernel_72012/1743738010.py\1743738010.py]8;;\:]8;id=567845;file:///tmp/ipykernel_72012/1743738010.py#48\48]8;;\

                    INFO     Running filter: check_flagged_words on text                           ]8;id=702184;file:///tmp/ipykernel_72012/1743738010.py\1743738010.py]8;;\:]8;id=116188;file:///tmp/ipykernel_72012/1743738010.py#48\48]8;;\

                    INFO     Running cleaner: remove_empty_lines on text                           ]8;id=555736;file:///tmp/ipykernel_72012/1743738010.py\1743738010.py]8;;\:]8;id=259800;file:///tmp/ipykernel_72012/1743738010.py#61\61]8;;\

                    INFO     Running cleaner: normalize_whitespace on text                         ]8;id=134950;file:///tmp/ipykernel_72012/1743738010.py\1743738010.py]8;;\:]8;id=830635;file:///tmp/ipykernel_72012/1743738010.py#61\61]8;;\

...

                    INFO     Running global filter: minhash_dedup                                  ]8;id=950836;file:///tmp/ipykernel_72012/1743738010.py\1743738010.py]8;;\:]8;id=439296;file:///tmp/ipykernel_72012/1743738010.py#85\85]8;;\

Indexing signatures...:   0%|          | 0/18011 [00:00<?, ?it/s]

Constructing graph...:   0%|          | 0/7757 [00:00<?, ?it/s]

Iterating over components...:   0%|          | 0/10557 [00:00<?, ?it/s]

[11/30/22 01:42:35] INFO     Final dataset size: 10557                                              ]8;id=923059;file:///tmp/ipykernel_72012/285196897.py\285196897.py]8;;\:]8;id=589576;file:///tmp/ipykernel_72012/285196897.py#24\24]8;;\

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()